In [2]:
import os, sys
import glob

import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import datetime as dt

from scipy import ndimage

import seaborn as sns

sns.set(style="white")
sns.set_context("paper", font_scale=2.5)
sys.path.append('../')

from data import geonexl1b
import models
import utils
from inference import domain_to_domain, load_model

import torchvision

In [33]:
exp_config_file = '/nobackupp10/tvandal/nex-ai-geo-translation/configs/Base-G16G17H8.yaml'
model, params = load_model(exp_config_file)
params

Loaded model from step: 199001


{'dataset': 'g16g17h8',
 'model_path': '/nobackupp10/tvandal/nex-ai-geo-translation/experiments/1/G16G17H8_16-16-16',
 'data': {'G16': {'data_url': 'file:///nobackupp10/tvandal/data/petastorm/G16_64x64_2km-large/',
   'bands': '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15',
   'shared': {'G17': '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15',
    'H8': '0,1,2,4,5,6,7,8,9,10,11,12,13,14,15'},
   'dim': 16},
  'G17': {'data_url': 'file:///nobackupp10/tvandal/data/petastorm/G17_64x64_2km-large/',
   'bands': '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15',
   'shared': {'G16': '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15',
    'H8': '0,1,2,4,5,6,7,8,9,10,11,12,13,14,15'},
   'dim': 16},
  'H8': {'data_url': 'file:///nobackupp10/tvandal/data/petastorm/H8_64x64_2km-large/',
   'bands': '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15',
   'shared': {'G16': '0,2,3,4,5,6,7,8,9,10,11,12,13,14,15',
    'G17': '0,2,3,4,5,6,7,8,9,10,11,12,13,14,15'},
   'dim': 16}},
 'num_workers': 8,
 'checkpoint_step': 5000,
 'log_iter': 250,
 'max_

In [34]:
goespaired = geonexl1b.L1bPaired('/nobackupp10/tvandal/nex-ai-geo-translation/data/Test/GOES16/GEONEX-L1G/', 
                                 '/nobackupp10/tvandal/nex-ai-geo-translation/data/Test/GOES17/GEONEX-L1G/',
                                 'G16', 'G17')
paired_files = goespaired.files(cachedir='.tmp').reset_index()
paired_files

g16_files = paired_files.pivot('v1', 'h1', 'file1')
g17_files = paired_files.pivot('v2', 'h2', 'file2')

In [31]:
files = g16_files

hsize = files.shape[1]
vsize = files.shape[0]

size = 600
data = np.empty((vsize*size, hsize*size, 16))
prediction = np.empty_like(data)
latent_space = np.empty((vsize*size, hsize*size, 64))
for ih in range(hsize):
    for iv in range(vsize):
        f = files.values[iv, ih]
        if isinstance(f, float):
            continue
        arr = geonexl1b.L1bFile(f, resolution_km=1.).load()
        data[iv*size:(iv+1)*size, ih*size:(ih+1)*size] = arr
        h8_synth, z = domain_to_domain(model, arr, 'G16', 'H8', latent=True)
        prediction[iv*size:(iv+1)*size, ih*size:(ih+1)*size] = h8_synth
        #latent_space[iv*size:(iv+1)*size, ih*size:(ih+1)*size] =  np.transpose(z, (1,2,0))

Loaded model from step: 199001


In [32]:
false_color = data[:,:,[2,1,0]]
mn = np.nanmin(false_color)
mx = np.nanmax(false_color)
false_color = (false_color - mn) / (mx-mn)

plt.imsave('G16G17_test_overlap.png', false_color)

true_color = prediction[:,:,[2,1,0]]
mn = np.nanmin(true_color)
mx = np.nanmax(true_color)
true_color = (true_color - mn) / (mx-mn)

plt.imsave('G16_to_RGB_test_overlap.png', true_color)


In [60]:
sensor = 'G17'
directory = '/nobackupp10/tvandal/nex-ai-geo-translation/data/Test/GOES17/GEONEX-L1B/'

goes = geonexl1b.GeoNEXL1b(directory, sensor)
goesfiles = goes.files(dayofyear=2)

pivot_files = goesfiles.pivot('v', 'h', 'file')

#hrange = list(range(44,60)) + [0,1,2,3]
#pivot_files = pivot_files[hrange]
pivot_files

h,58,59
v,,
16,/nobackupp10/tvandal/nex-ai-geo-translation/da...,/nobackupp10/tvandal/nex-ai-geo-translation/da...
17,/nobackupp10/tvandal/nex-ai-geo-translation/da...,/nobackupp10/tvandal/nex-ai-geo-translation/da...


In [61]:
hsize = pivot_files.shape[1]
vsize = pivot_files.shape[0]

size = 600
data = np.empty((vsize*size, hsize*size, 16))
prediction = np.empty_like(data)
latent_space = np.empty((vsize*size, hsize*size, 64))
for ih in range(hsize):
    for iv in range(vsize):
        f = pivot_files.values[iv, ih]
        if isinstance(f, float):
            continue
        arr = geonexl1b.L1bFile(f, resolution_km=1.).load()
        data[iv*size:(iv+1)*size, ih*size:(ih+1)*size] = arr
data.shape

(1200, 1200, 16)

In [62]:
false_color = data[:,:,[2,1,0]]
mn = 0. #np.nanmin(false_color)
mx = 1.6 #np.nanmax(false_color)*0.9

false_color[false_color > mx] = mx
false_color[false_color < mn] = mn

false_color = (false_color - mn) / (mx - mn)

plt.imsave(f'{sensor}_falsecolor.png', false_color.astype(np.float32), dpi=500)